In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import pandas as pd
import pathlib

In [3]:
# read in data

base_path = pathlib.Path.cwd().parent.parent / 'datasets'
inv_path = base_path / 'inv_urls.csv'
full_path = base_path / 'fulltext.pckl.gz'
meta_path = base_path / 'metadata.pckl.gz'

inv_urls = pd.read_csv(inv_path)
fulltext = pd.read_pickle(full_path, compression='gzip')
metadata = pd.read_pickle(meta_path, compression='gzip')

In [4]:
# controlled vocabulary list
vocab = ['death rite', 
         'burial site', 
         'burial mound',  
         'excavat', 
         'ceremon', 
         'shell bead', 
         'culturally unidentifiable human remains']

# list of phrases similar to 'unidentifiable human remains'
unident_vocab = ['culturally unidentifiable native american human remains',
                 'culturally unidentifiable native american remains',
                 'unidentifiable native american remains',
                 'unidentifiable native american',
                 'culturally unidentifiable human remains',
                 'unidentifiable human remains',
                 'culturally unidentifiable', 
                 'unidentifiable remains',
                 'unidentifiable',
                 'culturally unident',
                 'unident']

In [5]:
# create dictionary to 
# key is a phrase in unident_vocab
# value is list of all phrases in unident_vocab which are supersets of key
unident_dict = {x:[y for y in unident_vocab if (x in y) & (x!=y)] \
                                                    for x in unident_vocab}

In [6]:
# make a copy of fulltext, with all text lowercase
vocab_df = fulltext.copy()
vocab_df = vocab_df.applymap(str.lower)

In [7]:
# create binary column for each phrase in controlled vocabulary
for phrase in vocab:
    vocab_df[phrase] = vocab_df.messy_full.map(lambda x: phrase in x)
for phrase in unident_vocab:
    vocab_df[phrase] = vocab_df.messy_full.map(lambda x: phrase in x)

In [8]:
def context(phrase, text):
    
    # split text approximately into sentences
    sentences = text.split('.')
    
    # check each sentence for phrase
    for x in sentences:
        if phrase in x:
            # return sentence if phrase is in it
            return x
        
    # return "None" if phrase not found in any sentence
    return "None"

def vocab_info(phrase, parents):
    
    # get total number of documents containing phrase
    num_total = vocab_df[phrase].sum()
    
    # return if phrase does not appear in any documents
    if num_total == 0:
        return 0, 0, 0, 'None', 'None'
    
    # get indices of documents containing phrase
    inv = [x for x in list(vocab_df[vocab_df[phrase]==1].index) if x[0]=='I']
    repat = [x for x in list(vocab_df[vocab_df[phrase]==1].index) if x[0]=='R']
    
    # get number of documents containing phrase
    num_inv = len(inv)
    num_repat = len(repat)
    
    
    for parent in parents:
        if len(inv)!=0: 
            
            # get indices of documents containing specified superset
            parent_inv = [x for x in list(vocab_df[vocab_df[parent]==1].index) \
                                                                        if x[0]=='I']
            # only consider documents which contain the phrase without superset
            inv = set(inv) - set(parent_inv)
            
            if len(inv)==0:
                # phrase does not appear in any documents without superset
                inv_sample_context = 'same as \"{}\"'.format(parent)
                
        if len(repat)!=0: 
            # get indices of documents containing specified superset
            parent_repat = [x for x in list(vocab_df[vocab_df[parent]==1].index) \
                                                                        if x[0]=='R']
            # only consider documents which contain the phrase without superset
            repat = set(repat) - set(parent_repat)
            
            if len(repat)==0:
                # phrase does not appear in any documents without superset
                repat_sample_context = 'same as \"{}\"'.format(parent)
                
        if len(inv)==0 and len(repat)==0:
            # break loop if both index sets are empty
            break

    inv = list(inv)    
    repat = list(repat)
    
    if len(inv)!=0:
        # provide context sample that does not also contain superset
        inv_sample_context = context(phrase, vocab_df.loc[inv[-1], 'messy_full'])
    if len(repat)!=0:
        # provide context sample that does not also contain superset
        repat_sample_context = context(phrase, vocab_df.loc[repat[-1], 'messy_full'])
    return num_total, num_inv, num_repat, inv_sample_context, repat_sample_context

In [9]:
# create dataframe with info on phrases similar to 'unidentifiable human remains'
unident_results = pd.DataFrame(data={phrase: vocab_info(phrase, parents) \
                                     for phrase, parents in unident_dict.items()}, 
                             index=['Number of Total Documents', 
                                    'Number of Notices of Inventory Completion', 
                                    'Number of Notices of Intent to Repatriate', 
                                    'Sample Inventory Context', 
                                    'Sample Repatriation Context']) \
                    .T.sort_values('Number of Total Documents')

In [10]:
# create dataframe with info on controlled vocab phrases
vocab_results = pd.DataFrame(data={phrase: vocab_info(phrase, []) for phrase in vocab}, 
                             index=['Number of Total Documents', 
                                    'Number of Notices of Inventory Completion', 
                                    'Number of Notices of Intent to Repatriate', 
                                    'Sample Inventory Context', 
                                    'Sample Repatriation Context']).T

In [11]:
# display individual dataframes with full text samples
pd.set_option('display.max_colwidth', -1)
display(vocab_results)
display(unident_results)

,Number of Total Documents,Number of Notices of Inventory Completion,Number of Notices of Intent to Repatriate,Sample Inventory Context,Sample Repatriation Context
death rite,1703,1262,441,"3001 (3)(a), the 109 objects described above are reasonably believed to have been placed with or near individual human remains at the time of death or later as part of the death rite or ceremony","2 (d)(2)(ii), this cultural item is reasonably believed to have been placed with or near individual human remains at the time of death or later as part of the death rite or ceremony and is believed, by a preponderance of the evidence, to have been removed from a specific burial site of a native american individual"
burial site,606,172,434,"according to museum records, the donor speculated that a mass burial site, similar to those used by native americans, was within the general area where the human remains were unearthed","2 (d)(2)(ii), this cultural item is reasonably believed to have been placed with or near individual human remains at the time of death or later as part of the death rite or ceremony and is believed, by a preponderance of the evidence, to have been removed from a specific burial site of a native american individual"
burial mound,75,61,14,100-500) burial mound that was damaged by trenching in 1917,"\n in 1977, these items were recovered during legally authorized \nexcavations of portions of two burial mounds at the winnibigoshish dam \nsite (21 ic 4) near\n\n[[page 44924]]\n\nwinnibigoshish dam, chippewa national forest, itasca county, mn"
excavat,1282,1065,217,"at the time of excavation, the remains of only one adult individual were identified","\n in 1963, cultural items were removed from cremation features during excavations at site az u:13:9 asm, gila river indian reservation, pinal county, az, by arizona state museum staff member alfred e"
ceremon,2008,1272,736,"3001 (3)(a), the 109 objects described above are reasonably believed to have been placed with or near individual human remains at the time of death or later as part of the death rite or ceremony","2 (d)(2)(ii), this cultural item is reasonably believed to have been placed with or near individual human remains at the time of death or later as part of the death rite or ceremony and is believed, by a preponderance of the evidence, to have been removed from a specific burial site of a native american individual"
shell bead,404,290,114,"the associated funerary objects, including shell beads and shell gorgets, stone and bone tools, ornaments, trade beads and metal trade objects were also excavated from these burials and were documented in reports generated in the 1980s by university of tennessee and university of georgia professional staff","the 5,508 unassociated funerary objects (207 inventory numbers) are 2 abalone shell containers; 2 abraders; 1 arrow straightener; 36 arrow points (3 arrow points, 5 chalcedony, 4 chalcedony and chert, 6 obsidian, 1 red carnelian, 5 stone, 12 obsidian and chalcedony); 2 asphaltum pieces; 3 balls (1 granite, 1 sandstone, and 1 wood); 9 basket fragments, 1 bag with tiny beads and fragments and 5,156 individual beads (15 clam shell beads, 10 pismo clam shell beads, 4 amethystine beads, 2,010 trade beads, 1 serpetine bead, 51 steatite beads, 2 stone beads, 22 red and white beads, 307 olivella beads, 365 red beads, 2,065 blue beads, 42 black beads, 113 green beads, 111 white, 1 yellow, 19 amber beads, 3 pink beads, 3 miscellaneous beads, 1 unknown bead, 3 soapstone beads, 1 crystal beads, 2 shell beads, and 5 tubular beads); 2 boiling stones; 1 glass bottle neck; 3 bowls (1 sandstone, 1 stone, and 1seatite); 20 bowl fragments (5 steatite, 7 sandstone, 1 wooden, and 7 soapstone); 1 brush; 1 bull roarer fragment; 10 buttons (8 brass and 2 metal); 2 charmstones; 1 chert chalcedony; 2 china pitchers; 1 china saucer; 5 bird claws; 1 comal; 2 cooking stones; 3 crosses (2 metal crosses and 1 silver cross); 1 crystal; 2 crystal and mica fragments; 5 

,Number of Total Documents,Number of Notices of Inventory Completion,Number of Notices of Intent to Repatriate,Sample Inventory Context,Sample Repatriation Context
culturally unidentifiable native american remains,0,0,0,None,None
unidentifiable native american remains,0,0,0,None,None
unidentifiable remains,0,0,0,None,None
culturally unidentifiable native american human remains,49,48,1,"in september 2012, the university of washington, department of anthropology, requested that the secretary, through the native american graves protection and repatriation review committee, recommend the proposed transfer of control of the culturally unidentifiable native american human remains in this notice to the confederated tribes and bands of the yakama nation; confederated tribes of the chehalis reservation; confederated tribes of the colville reservation; confederated tribes of the umatilla indian reservation (previously listed as the confederated tribes of the umatilla reservation, oregon); confederated tribes of the warm springs reservation of oregon; puyallup tribe of the puyallup reservation; samish indian nation (previously listed as the samish indian tribe, washington); stillaguamish tribe of indians of washington (previously listed as the stillaguamish tribe of washington); suquamish indian tribe of the port madison reservation; tulalip tribes of washington (previously listed as the tulalip tribes of the tulalip reservation, washington); upper skagit indian tribe; and the wanapum band of priest rapids, a non-federally recognized indian group (which together, comprise the washington state inter-tribal consortium)","\n\n in march 1999, the new hampshire division of historical resources, acting on behalf of the university of new hampshire and three other museums, presented a disposition proposal to the nagpra review committee for culturally unidentifiable native american human remains"
unidentifiable native american,49,48,1,"same as ""culturally unidentifiable native american human remains""","same as ""culturally unidentifiable native american human remains"""
culturally unidentifiable human remains,126,124,2,"16, the secretary of the interior may make a recommendation for a transfer of control of culturally unidentifiable human remains","\n in july 2003, the flandreau santee sioux tribe of south dakota submitted a request to the archaeological research center for repatriation of the culturally unidentifiable human remains and funerary objects from eastern, central, and northwestern south dakota, and southeastern montana, including the 14 unassociated funerary objects from the daugherty mounds and scalp creek sites, on behalf of the cheyenne river sioux tribe of the cheyenne river reservation, south dakota; crow creek sioux tribe of the crow creek reservation, south dakota; flandreau santee sioux tribe of south dakota; iowa tribe of oklahoma; lower sioux indian community in the state of minnesota; northern cheyenne tribe of the northern cheyenne indian reservation, montana; oglala sioux tribe of the pine ridge reservation, south dakota; omaha tribe of nebraska; otoe-missouria tribe of indians, oklahoma; prairie island indian community in the state of minnesota; rosebud sioux tribe of the rosebud indian reservation, south dakota; sac & fox tribe of the mississippi in iowa; santee sioux nation, nebraska; sisseton-wahpeton oyate of the lake traverse reservation, south dakota; three affiliated tribes of the fort berthold reservation, north dakota; upper sioux indian community, minnesota; and yankton sioux tribe of south dakota"
unidentifiable human remains,126,124,2,"same as ""culturally unidentifiable human remains""","same as ""culturally unidentifiable human remains"""
culturally unidentifiable,158,155,3,these human remains come from an unknown location and were collected under unknown circumstances and are now re-classified as culturally unidentifiable,"same as ""culturally unidentifiable human remains"""
culturally unident,159,156,3,